In [ ]:
#model parameters
import numpy as np
import samples as ls
import math
import model_types


# networks used for thesis:
# RNNs:
# 0/1/2_9
# 10/11_34, 12_36 (34 used sgd, 36 adam, not really a difference)
# DNNs:
# 11_39 (sigmoid)
# 11_45 (relu)

# quick overview of the modes:
# 1. load_test: Load the test data for final evaluation. It is only used for testing and not training
# 2. predict_mode: No training. Load all data uncut/without unrolling. 
#                  Makes recurrent networks stateful (they remember their state between runs). This enables the live demo.
# 3. None of the above: Training mode, used to train the networks.


#--------- TRAINING MODES --------------
#load final test data for final evaluation
load_test = False

#enable to make RNNs stateful (save state between batches) and to not unroll training data for whole sample comparison
predict_mode = True

#load the network from saved file (needs to be False if network is not yet created)
load_rnn = True if not predict_mode else False
#only load the weights, not the architecture
load_only_weights = False if not load_rnn else False
#model to use (num=architecture, type=labels/task)
model_num = 9
model_type = 2

train_network = False if not load_test else False
live_demo = False


#--------- MODEL DATA --------------
model_id = '%s_%s' % (model_type, model_num)
model_path = 'tmp/keras/nn-%s.hdf5' % model_id
weights_path = 'tmp/keras/nn-%s_weights.hdf5' % model_id
labels,sample_ids,filename = model_types.get_labels(model_type, load_test)


#--------- MODEL HYPERPARAMETERS --------------
accel,gyro,compass = 'xyz','xyz',''
overlap_step = 1
use_labels_data = False #-> if true, output at each timestep is compared (should be true for first two tasks)
use_lstm = True
use_sgd = False #-> if false, adamoptimizer is used
use_rnn = True #use a recurrent cell
input_steps = 50 #both set to 50 as the default setting for the angle network
unroll_steps = 50
activation = 'sigmoid'
load_samples = True
dropout_prob = 0 #for recurrent cells
dropout_dense = False #for dense layers
complete_unroll = False


#--------- MODEL DEFINITIONS --------------
elif model_num == 9:
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_labels_data = True
    complete_unroll = True
    hidden_layers = [14]
elif model_num == 34:
    use_rnn = True
    use_lstm = True
    use_labels_data = False
    load_samples = False
    use_sgd = True
    hidden_layers = [24]
elif model_num == 36:
    use_rnn = True
    use_lstm = True
    use_labels_data = False
    load_samples = False
    hidden_layers = [24]
elif model_num == 39:
    use_rnn = False
    activation = 'sigmoid'
    hidden_layers = [50,24]
elif model_num == 45:
    use_rnn = False
    activation = 'relu'
    hidden_layers = [24,24,24,24,12]
else:
    raise ValueError('Unknown model')
    
classes = len(labels)

#predict mode/test mode -> dont unroll for task 1 and 2
if predict_mode:
    input_steps = None
    unroll_steps = None
    overlap_step = 1
#however, for angle task always unroll
if model_type >= 10: 
    load_samples = False
    input_steps = 50 #both set to 50 as the default setting for the angle network
    unroll_steps = 50

#percentage used for validation data
test_rate = 0.1 if not load_test else 1

#learning rate for sgd if used
learning_rate = 0.01

print('model: %s (type %s, num %s)' % (model_id,model_type,model_num))
print('%s classes' % (classes))

In [ ]:
#--------- PREPARE DATA --------------
#load samples
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)
    
    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)

    samples.convert_to_input(size = None, accel=accel, gyro=gyro, compass=compass)

samples.unroll(unroll_steps, overlap_step, complete=complete_unroll)
samples.convert_to_onehot()
samples.split_test(test_rate, random=False)
if not use_rnn: samples.flatten()

input_dim = samples.input_dim
print("dim: %s" % input_dim)
    
#prepare training data
xtrain,ytrain,seqlen = samples.get_all(padding=False, use_labels_data=use_labels_data)
xtest,ytest,seqlentest = samples.get_test(padding=False, use_labels_data=use_labels_data)

if unroll_steps is None and not predict_mode and not load_test:
    from keras.preprocessing import sequence
    xtrain = sequence.pad_sequences(xtrain)
    xtest = sequence.pad_sequences(xtest)
    #not efficient but whatevs
    if len(xtrain[0]) > len(xtest[0]):
        input_steps = len(xtrain[0])
        xtest = sequence.pad_sequences(xtest, maxlen=input_steps)
    elif len(xtrain[0]) < len(xtest[0]):
        input_steps = len(xtest[0])
        xtrain = sequence.pad_sequences(xtrain, maxlen=input_steps)
    else:
        input_steps = len(xtrain[0])
    
    ytrain = sequence.pad_sequences(ytrain, maxlen=input_steps)
    ytest = sequence.pad_sequences(ytest, maxlen=input_steps)

In [ ]:
#--------- PREPARE MODELS --------------
#create/load model
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Masking
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

if load_rnn and not load_only_weights:
    model = load_model(model_path)
    model.load_weights(weights_path)
else:
    model = Sequential()
    if unroll_steps is None and not predict_mode:
        #variable length -> mask paddings
        #(not implemented for predict_mode)
        model.add(Masking(input_shape=(input_steps, input_dim)))
        
        if use_lstm:
            model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       return_sequences=use_labels_data))
        else:
            model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       return_sequences=use_labels_data))
                     
        if use_labels_data:
            for k in hidden_layers:
                model.add(TimeDistributed(Dense(k, activation=activation)))
            model.add(TimeDistributed(Dense(classes, activation='softmax')))
        else:
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation='softmax'))
        
    elif use_rnn:
        #recurrent structure: one recurrent cell followed by time distributed layers
        if use_lstm:
            if predict_mode:
                model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       batch_input_shape=(1, input_steps, input_dim), return_sequences=use_labels_data, stateful=True))
            else:
                #hidden_layers.append(classes)
                #model.add(LSTM(hidden_layers.pop(0), activation='softmax', dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                #       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
                model.add(LSTM(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
        else:
            if predict_mode:
                model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       batch_input_shape=(1, input_steps, input_dim), return_sequences=use_labels_data, stateful=True))
            else:
                model.add(SimpleRNN(hidden_layers.pop(0), dropout=dropout_prob, recurrent_dropout=dropout_prob, 
                       input_shape=(input_steps, input_dim), return_sequences=use_labels_data))
                     
        if use_labels_data:
            for k in hidden_layers:
                model.add(TimeDistributed(Dense(k, activation=activation)))
            model.add(TimeDistributed(Dense(classes, activation='softmax')))
        else:
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
            model.add(Dense(classes, activation='softmax'))
    else:
        #purely DNN
        if not hidden_layers: #(no hidden layers)
            model.add(Dense(classes, activation='softmax', input_shape=(input_steps*input_dim,)))
        else:
            model.add(Dense(hidden_layers.pop(0), activation=activation, input_shape=(input_steps*input_dim,)))
            for k in hidden_layers:
                model.add(Dense(k, activation=activation))
                if dropout_dense: model.add(Dropout(0.2))
            model.add(Dense(classes, activation='softmax'))
   
        
    

    if use_sgd: opt = optimizers.SGD(lr=learning_rate)
    else: opt = 'adam'
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    if predict_mode or load_only_weights:
        #do not save model in predict_mode!
        model.load_weights(weights_path)
    else:
        model.save(model_path)
        model.save_weights(weights_path)
        

print(model.summary())

In [ ]:
#--------- TRAINING/PREDICTION FUNCTIONS --------------
if not predict_mode:
    #only relevant when training, not in predict mode
    """
    Trains the network for the given epochs and batch_size. The network with the best validation accuracy is saved.
    @param epochs: how many epochs to train for
    @param batch_size: the batch size to use
    """
    def train(epochs, batch_size):
        mcp = ModelCheckpoint(weights_path, monitor='val_acc', save_best_only=True, save_weights_only=True)
        model.fit(xtrain, ytrain,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(xtest, ytest),
                  callbacks=[mcp])
        score, acc = model.evaluate(xtest, ytest, batch_size=len(xtest))
        print('Test score: %s' % score)
        print('Test accuracy: %s' % acc)
        #model.save(model_path)
        #model.save_weights(weights_path)
        
"""
Test all available data (both train and validation set).
@param batch_size: feed batches of this size to reduce memory needed
"""    
def test_total(batch_size=None):
    if batch_size == None:
        acc1 = model.evaluate(xtrain, ytrain, batch_size=len(xtrain), verbose=0)[1]
        acc2 = model.evaluate(xtest, ytest, batch_size=len(xtest), verbose=0)[1]
    else:
        acc1 = 0
        num = len(xtrain)
        for i in range(0,num,batch_size):
            size = min(batch_size, len(xtrain)-i)
            xtrain2 = xtrain[i:i+size]
            ytrain2 = ytrain[i:i+size]
            acc1 += (float(size)/num) * model.evaluate(xtrain2, ytrain2, batch_size=size, verbose=0)[1]

        acc2 = 0
        num = len(xtest)
        for i in range(0,num,batch_size):
            size = min(batch_size, len(xtest)-i)
            xtrain2 = xtest[i:i+size]
            ytrain2 = ytest[i:i+size]
            acc2 += (float(size)/num) * model.evaluate(xtrain2, ytrain2, batch_size=size, verbose=0)[1]
    acc = acc1*(1.0-test_rate) + acc2*test_rate
    print('Train accuracy: %s' % acc1)
    print('Test accuracy: %s' % acc2)
    print('Total accuracy: %s' % acc)

"""
Evaluate the model using the test/validation data.
"""
def test():
    score, acc = model.evaluate(xtest, ytest, batch_size=len(xtest))
    print('Test score: %s' % score)
    print('Test accuracy: %s' % acc)


"""
Calculates the per-label accuracies.
@param test: if true, the test/validation set is used
"""
def per_label_acc(test=True):
    if test:
        x,y = xtest,ytest
    else:
        x,y = xtrain,ytrain
    accs = [0] * classes
    counts = [0] * classes

    print_step = len(y)/10
    for i in range(len(y)):
        acc = model.evaluate(np.array([x[i]]), np.array([y[i]]), batch_size=1, verbose=0)[1]
        c = 0
        if use_labels_data:
            for y_oh in y[i]:
                c = np.argmax(y_oh)
                accs[c] += acc
                counts[c] += 1
        else:
            c = np.argmax(y[i])
            accs[c] += acc
            counts[c] += 1
        if i%print_step==0:
            print('Progress: %s' % (float(i)/len(y)))

    acc_total = 0
    for i in range(classes):
        acc = accs[i]/counts[i]
        print('%sx %s: %s' % (counts[i],labels[i],acc))
        acc_total += acc*1.0/classes
    print('Total test acc: %s' % acc_total)

"""
Interface to make feeding data through a recurrent network simpler.
@param s: list of data points, if feeding only a single timestep, this needs to be a list with one element
@param as_prob: return the output as probabilities instead of argmax
@return a list with the output of the network for each of the input steps
"""
def feed_seq(s, as_prob=False):
    if normalize:
        for step in s:
            norm(step)
    res = model.predict(np.array([s]),batch_size=1)[0]
    resarg = []
    for r in res:
        if as_prob: resarg.append(r)
        else: resarg.append(np.argmax(r))
    return resarg


In [ ]:
#small test: compare output of network for one example sample (for each label)
if predict_mode:
    if False:
        import time
        start = time.time()
        for i in range(10000):
            z = [0,0,0,0,0,0]
            v = [z]*1
            feed_seq(v)
        dt = time.time() - start
        print('%s hz' % (1/(dt/10000)))

    if True:
        for key in samples.test:
            model.reset_states()
            xdata = samples.test[key][0][0]
            ydata = samples.labels_test[key][0][0]

            res = feed_seq(xdata)

            res_predicted = res
            res_expected = []
            acc = 0
            for i in range(len(res)):
                res_expected.append(np.argmax(ydata[i]))
                if res_predicted[len(res_expected)-1] == res_expected[-1]:
                    acc += 1.0
            print('%s: %s' % (key, acc/len(res)))
            print(res_predicted)
            print(res_expected)

In [ ]:
#display how many samples are available
if True:
    for key in labels:
        print('%s: %s / %s' % (key, len(samples.data[key]), len(samples.test[key])))

In [ ]:
#train the network
if train_network and not predict_mode:
    train(300,40)

In [ ]:
#evaluate model
if True:
    per_label_acc()

In [ ]:
#live demo
%env ROS_HOSTNAME=zinunb
%env ROS_IP=$(hostname -I)
%env ROS_MASTER_URI=http://pi:11311
        
if live_demo and predict_mode:
    import imu_listener as il
    #reset()
    listener = il.imu_listener(feed_seq,labels,1,1,record=False)
    #listener.save_history()
    print('Exited')